# Rule Recommendation Model for Automation

A sector agnostic GNN-based custom rules recommendation for device management using preset internal links for personalized automation.

This sample notebook shows you how to execute clean-sentiment-classification-labels Algorithm from AWS Marketplace

 **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

### Pre-requisites:

1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
2. Ensure that IAM role used has **AmazonSageMakerFullAccess**
3. Some hands-on experience using Amazon SageMaker.
4. To use this algorithm successfully, ensure that:  
    A. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the            AWS account used:  
&nbsp;&nbsp;&nbsp;&nbsp;            a. **aws-marketplace:ViewSubscriptions**  
&nbsp;&nbsp;&nbsp;&nbsp;            b. **aws-marketplace:Unsubscribe**  
&nbsp;&nbsp;&nbsp;&nbsp;            c. **aws-marketplace:Subscribe**  
    B. or your AWS account has a subscription to For Seller to update: Rule Recommendation Model for Automation.


### Contents

1. [Subscribe to the algorithm]()
2. [Prepare dataset]()  
    A. [Dataset format expected by the algorithm]()  
    B. [Configure dataset]()  
    C. [Upload datasets to Amazon S3]()  
3. [Execute optimization model]()  
    A. [Set up environment]()    
    B. [Execute model]()  
    C. [Visualize Output]()  
4. [Clean-up]()  
    A. [Unsubscribe to the listing (optional))]()

### 1. Subscribe to the algorithm

To subscribe to the algorithm:  
&nbsp; 1. Open the algorithm listing page **Rule Recommendation Model for Automation**  
&nbsp; 2. On the AWS Marketplace listing, click on **Continue to subscribe** button.  
&nbsp; 3. On the **Subscribe to this software** page, review and click on "**Accept Offer**" if you agree with EULA, pricing, and support terms.  
&nbsp; 4. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn**. This is the algorithm ARN that you need to specify while training a custom ML model. Copy the ARN corresponding to your region and specify the same in the following cell.

In [1]:
algo_arn = "your_algo_arn"

### 2. Prepare dataset

In [2]:
import os
import json 
import uuid
import boto3
import pickle
import base64
import tarfile
from pprint import pprint
from PIL import Image

import numpy as np
import pandas as pd

import urllib.request
from urllib.parse import urlparse

import sagemaker as sage
from sagemaker import ModelPackage
from sagemaker import get_execution_role



sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


#### A. Dataset format expected by the algorithm

The algorithm needs two csv files:
1. train_rule.csv containing details aboutthe existing rules including userid,	Trigger device,	Trigger deviceid	Trigger state,	Trigger state id,Action	action id,Action Device, Action Device id and rule.
2. train_devices.csv containing all the devices with user id and device id 

#### B. Configure dataset

In [3]:
training_dataset="Input/training/input_zip.zip"

#### C. Upload datasets to Amazon S3

In [4]:
role = get_execution_role()

In [ ]:
sagemaker_session = sage.Session()

bucket = sagemaker_session.default_bucket()
bucket

In [ ]:
# training input location
common_prefix = "rule-recommendation-engine-for-automation"
training_input_prefix = common_prefix + "/training-input-data"
TRAINING_WORKDIR = "Input" #Input directory in Jupyter Server
training_input = sagemaker_session.upload_data(TRAINING_WORKDIR, key_prefix=training_input_prefix) #uploads data from jupyter server to S3
print("Training input uploaded to " + training_input)

### 3. Execute the training process

Now that dataset is available in an accessible Amazon S3 bucket, we are ready to execute a training pipeline to get clean sentiment class labels using clean-sentiment-classification-labels Algorithm.

#### 3.1 Set up environment

In [7]:
output_location = 's3://{}/rule-recommendation-engine-for-automation/{}'.format(bucket, 'Output')

#### 3.2 Execute model

For information on creating an Estimator object, see [documentation](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html)

In [8]:
training_instance_type="ml.m5.4xlarge"

In [ ]:
#Create an estimator object for running a training job
estimator = sage.algorithm.AlgorithmEstimator(
    algorithm_arn=algo_arn,
    base_job_name="rule-recommendation-engine-for-automation",
    role=role,
    train_instance_count=1,
    train_instance_type=training_instance_type,
    input_mode="File",
    output_path=output_location,
    sagemaker_session=sagemaker_session,
    instance_count=1,
    instance_type=training_instance_type
)

#Run the training job.
estimator.fit({"training": training_input})

INFO:sagemaker:Creating training-job with name: rule-recommendation-engine-for-automati-2024-08-05-04-02-13-072


2024-08-05 04:02:16 Starting - Starting the training job...
2024-08-05 04:02:31 Starting - Preparing the instances for training...
2024-08-05 04:03:00 Downloading - Downloading the training image......
2024-08-05 04:04:00 Training - Training image download completed. Training in progress.....traininf file sent to main
Start loading data...
Data loaded
Number of rules: 852072
Number of devices: 1354098
Filtering rules with fewer than 25 occurrences
Aggregating devices by user
Aggregating rules by user
#0150it [00:00, ?it/s]#0151it [00:00,  5.05it/s]#0151462it [00:00, 6125.09it/s]#0152936it [00:00, 9350.85it/s]#0154567it [00:00, 11773.42it/s]#0156197it [00:00, 13275.81it/s]#0157821it [00:00, 14229.45it/s]#0159455it [00:00, 14893.54it/s]#01511090it [00:00, 15343.03it/s]#01512726it [00:00, 15652.70it/s]#01514355it [00:01, 15845.12it/s]#01515990it [00:01, 15996.44it/s]#01517622it [00:01, 16093.49it/s]#01519257it [00:01, 16169.70it/s]#01520886it [00:01, 16205.21it/s]#01522522it [00:01, 16249

#015Epoch 1/1:   0%|          | 0/424 [00:00<?, ?batch/s]#015Epoch 1/1:   0%|          | 0/424 [00:00<?, ?batch/s, Loss=0.695]#015Epoch 1/1:   0%|          | 1/424 [00:00<01:18,  5.38batch/s, Loss=0.695]#015Epoch 1/1:   0%|          | 1/424 [00:00<01:18,  5.38batch/s, Loss=0.685]#015Epoch 1/1:   0%|          | 2/424 [00:00<01:05,  6.41batch/s, Loss=0.685]#015Epoch 1/1:   0%|          | 2/424 [00:00<01:05,  6.41batch/s, Loss=0.678]#015Epoch 1/1:   1%|          | 3/424 [00:00<01:03,  6.67batch/s, Loss=0.678]#015Epoch 1/1:   1%|          | 3/424 [00:00<01:03,  6.67batch/s, Loss=0.668]#015Epoch 1/1:   1%|          | 4/424 [00:00<01:01,  6.86batch/s, Loss=0.668]#015Epoch 1/1:   1%|          | 4/424 [00:00<01:01,  6.86batch/s, Loss=0.66] #015Epoch 1/1:   1%|          | 5/424 [00:00<00:59,  7.05batch/s, Loss=0.66]#015Epoch 1/1:   1%|          | 5/424 [00:00<00:59,  7.05batch/s, Loss=0.646]#015Epoch 1/1:   1%|▏         | 6/424 [00:00<00:58,  7.20batch/s, Loss=0.646]#015Epoch 1/1:   1%|▏       

]#015Epoch 1/1:  25%|██▍       | 105/424 [00:14<00:44,  7.11batch/s, Loss=0.177]#015Epoch 1/1:  25%|██▍       | 105/424 [00:14<00:44,  7.11batch/s, Loss=0.171]#015Epoch 1/1:  25%|██▌       | 106/424 [00:14<00:44,  7.10batch/s, Loss=0.171]#015Epoch 1/1:  25%|██▌       | 106/424 [00:14<00:44,  7.10batch/s, Loss=0.172]#015Epoch 1/1:  25%|██▌       | 107/424 [00:14<00:44,  7.08batch/s, Loss=0.172]#015Epoch 1/1:  25%|██▌       | 107/424 [00:15<00:44,  7.08batch/s, Loss=0.215]#015Epoch 1/1:  25%|██▌       | 108/424 [00:15<00:44,  7.03batch/s, Loss=0.215]#015Epoch 1/1:  25%|██▌       | 108/424 [00:15<00:44,  7.03batch/s, Loss=0.148]#015Epoch 1/1:  26%|██▌       | 109/424 [00:15<00:44,  7.10batch/s, Loss=0.148]#015Epoch 1/1:  26%|██▌       | 109/424 [00:15<00:44,  7.10batch/s, Loss=0.129]#015Epoch 1/1:  26%|██▌       | 110/424 [00:15<00:44,  7.11batch/s, Loss=0.129]#015Epoch 1/1:  26%|██▌       | 110/424 [00:15<00:44,  7.11batch/s, Loss=0.16] #015Epoch 1/1:  26%|██▌       | 111/424 [00:15<00:4

s, Loss=0.0957]#015Epoch 1/1:  48%|████▊     | 202/424 [00:28<00:31,  7.04batch/s, Loss=0.0952]#015Epoch 1/1:  48%|████▊     | 203/424 [00:28<00:31,  7.04batch/s, Loss=0.0952]#015Epoch 1/1:  48%|████▊     | 203/424 [00:28<00:31,  7.04batch/s, Loss=0.112] #015Epoch 1/1:  48%|████▊     | 204/424 [00:28<00:31,  7.04batch/s, Loss=0.112]#015Epoch 1/1:  48%|████▊     | 204/424 [00:28<00:31,  7.04batch/s, Loss=0.122]#015Epoch 1/1:  48%|████▊     | 205/424 [00:28<00:31,  6.89batch/s, Loss=0.122]#015Epoch 1/1:  48%|████▊     | 205/424 [00:28<00:31,  6.89batch/s, Loss=0.105]#015Epoch 1/1:  49%|████▊     | 206/424 [00:28<00:31,  6.94batch/s, Loss=0.105]#015Epoch 1/1:  49%|████▊     | 206/424 [00:29<00:31,  6.94batch/s, Loss=0.0979]#015Epoch 1/1:  49%|████▉     | 207/424 [00:29<00:31,  6.99batch/s, Loss=0.0979]#015Epoch 1/1:  49%|████▉     | 207/424 [00:29<00:31,  6.99batch/s, Loss=0.102] #015Epoch 1/1:  49%|████▉     | 208/424 [00:29<00:30,  7.11batch/s, Loss=0.102]#015Epoch 1/1:  49%|████▉     |

1<00:18,  7.16batch/s, Loss=0.0782]#015Epoch 1/1:  69%|██████▉   | 294/424 [00:41<00:18,  7.16batch/s, Loss=0.101] #015Epoch 1/1:  70%|██████▉   | 295/424 [00:41<00:18,  7.11batch/s, Loss=0.101]#015Epoch 1/1:  70%|██████▉   | 295/424 [00:41<00:18,  7.11batch/s, Loss=0.0705]#015Epoch 1/1:  70%|██████▉   | 296/424 [00:41<00:17,  7.15batch/s, Loss=0.0705]#015Epoch 1/1:  70%|██████▉   | 296/424 [00:41<00:17,  7.15batch/s, Loss=0.0922]#015Epoch 1/1:  70%|███████   | 297/424 [00:41<00:17,  7.18batch/s, Loss=0.0922]#015Epoch 1/1:  70%|███████   | 297/424 [00:41<00:17,  7.18batch/s, Loss=0.0673]#015Epoch 1/1:  70%|███████   | 298/424 [00:41<00:17,  7.15batch/s, Loss=0.0673]#015Epoch 1/1:  70%|███████   | 298/424 [00:41<00:17,  7.15batch/s, Loss=0.0963]#015Epoch 1/1:  71%|███████   | 299/424 [00:41<00:17,  7.15batch/s, Loss=0.0963]#015Epoch 1/1:  71%|███████   | 299/424 [00:42<00:17,  7.15batch/s, Loss=0.0758]#015Epoch 1/1:  71%|███████   | 300/424 [00:42<00:17,  7.21batch/s, Loss=0.0758]#015Ep

Epoch 1/1, Average Validation Loss: 0.07658720958345341
Saving model..
Testing the model /opt/ml/model/checkpoint_epoch_1_valloss_0.07658720958345341.ckpt
Initializing LinkPredictor with num_trigger_states=45, num_actions=47
Model loaded
Training complete

2024-08-05 04:08:45 Uploading - Uploading generated training model
2024-08-05 04:08:45 Completed - Training job completed
Training seconds: 355
Billable seconds: 355


See this [blog-post](https://aws.amazon.com/blogs/machine-learning/easily-monitor-and-visualize-metrics-while-training-models-on-amazon-sagemaker/) for more information how to visualize metrics during the process. You can also open the training job from [Amazon SageMaker console](https://us-east-2.console.aws.amazon.com/sagemaker/home?region=us-east-2#/jobs/) and monitor the metrics/logs in **Monitor** section.

In [ ]:
#output is available on following path
estimator.output_path

#### 3.3 Inspect the Output in S3

In [ ]:
parsed_url = urlparse(estimator.output_path)
bucket_name = parsed_url.netloc
file_key = parsed_url.path[1:]+'/'+estimator.latest_training_job.job_name+'/output/'+"model.tar.gz"

s3_client = sagemaker_session.boto_session.client('s3')
response = s3_client.get_object(Bucket = sagemaker_session.default_bucket(), Key = file_key) 

In [ ]:
bucketFolder = estimator.output_path.rsplit('/')[3] +'/Output/'+ estimator.latest_training_job.job_name+'/output/'+"model.tar.gz"

### 4. Deploy model and verify results

Now you can deploy the model for performing real-time inference.

In [ ]:
role = get_execution_role()

sagemaker_session = sage.Session()

bucket=sagemaker_session.default_bucket()
bucket

In [ ]:
model_name='rule-recommendation-engine-for-automation'

content_type='text/csv'

real_time_inference_instance_type='ml.p3.2xlarge'
batch_transform_inference_instance_type='ml.p3.2xlarge'

#### 4.1 Deploy trained model

In [ ]:
from sagemaker.serializers import CSVSerializer

predictor = estimator.deploy(1, real_time_inference_instance_type)

#### 4.2 Create input payload

In [ ]:
file_name = 'Input/inference/test_sam.csv'

#### 4.3 Perform real-time inference

In [ ]:
output_file_name = 'output/inf_output.csv'

In [ ]:
!aws sagemaker-runtime invoke-endpoint \
    --endpoint-name $predictor.endpoint_name \
    --body fileb://$file_name \
    --content-type $content_type \
    --region $sagemaker_session.boto_region_name \
    $output_file_name

{
    "ContentType": "text/csv; charset=utf-8",
    "InvokedProductionVariant": "AllTraffic"
}


#### 4.4 Check output

In [3]:
output= pd.read_csv('output/inf_output.csv')
output.head()

,user_id,Trigger Device,Trigger Device ID,Action Device,Action Device ID,Trigger State,Trigger State ID,Action,Action ID,Rule
0,20017,ContactSensor,977316,MotionSensor,1407198,Detects motion,9,Upload a short video to the cloud,46,977316_9_46_1407198
1,20017,ContactSensor,977316,MotionSensor,1407198,Detects motion,9,Turn on notifications,38,977316_9_38_1407198
2,20017,ContactSensor,977316,MotionSensor,1407198,Detects motion,9,Turn on motion detection,37,977316_9_37_1407198
3,20017,ContactSensor,977316,MotionSensor,1407198,Detects motion,9,Turn on the siren,42,977316_9_42_1407198
4,20017,ContactSensor,977316,MotionSensor,1407198,Detects a person,6,Turn on notifications,38,977316_6_38_1407198


### 5. Clean-up

In [ ]:
predictor.delete_endpoint(delete_endpoint_config=True)

#### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the algorithm, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://us-east-2.console.aws.amazon.com/sagemaker/home?region=us-east-2#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model.

#### Steps to unsubscribe to product from AWS Marketplace:

&nbsp; 1. Navigate to **Machine Learning tab** on **Your Software subscriptions page**.  
&nbsp; 2. Locate the listing that you want to cancel the subscription for, and then choose **Cancel Subscription** to cancel the subscription.